<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT23/blob/main/template-report-lab-X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Iterative Methods**
**Teo Nordström**

# **Abstract**

This file contains the solutions to the three mandatory problems from Lab2 in DD2363, in addition to the solution to one of the optional problems. It is based upon pseudocode and info found in *Methods in Computational Science* by Johan Hoffman (2021)

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license.

In [48]:
"""This file is based on a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# TEMPLATE INFO:
# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

# CODE INFO:
# Code written by Teo Nordström 2024, no license.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

These are the neccessary modules for everything in this file to work.

Also set up some functions from a previous assignment.

In [49]:
from google.colab import files

import numpy as np
import math

def modified_gram_schmidt_iteration(A:np.ndarray):
    n = len(A)
    Q = np.zeros((n, n))
    R = np.zeros((n, n))
    for j in range(n):
        v = A[:, j]
        for i in range(j):
            R[i, j] = np.dot(Q[:, i], v)
            v = v - R[i, j] * Q[:, i]
        R[j, j] = np.linalg.norm(v)
        Q[:, j] = v / R[j, j]
    return Q, R


def backward_substitution(U:np.ndarray, b:np.ndarray):
    n = U.shape[1]
    x = np.zeros(n)
    x[n-1] = b[n-1] / U[n-1, n-1]
    for i in range(n-2, -1, -1):
        sum = 0
        for j in range(i+1, n):
            sum += U[i, j] * x[j]
        x[i] = (b[i] - sum) / U[i, i]
    return x


def direct_solver(A:np.ndarray, b:np.ndarray):
    Q, R = modified_gram_schmidt_iteration(A)
    y = np.transpose(Q).dot(b)
    x = backward_substitution(R, y)
    return x


# **Introduction**

All solutions will be partially or entirely based upon the book *Methods in Computational Science* by Johan Hoffman (2021). In the text, it will be referred to as the "course book".

# Jacobi Iteration
Jacobi Iteration is a version of Richardson Iteration that uses matrix splitting and precondition to find $x$ in a problem $Ax = n$. It is an iterative solution meaning it for each iteration should get closer to the answer, as long as $||I - D^{-1}A|| < 1||$ (where $D$ is the diagonal of $A$), since otherwise it will not converge. It it is more efficient than normal Richardson Iteration when matrix A is diagonally dominant.

# Gauss-Seidel Iteration
Gauss-Seidel Iteration is another version of Richardson Iteration that uses matrix splitting and precondition. Here, instead of using the diagonal $D$ we use the lower triangular matrix $L$ to split up $A$. $L$ is just $A$ with all non-lower-triangular values zeroed out. The convergence criterion here is $||I - L^{-1}A|| < 1||$ which once again means it works on diagonally dominant matrices.

# Newton's Method for Scalar Nonlinear Equation
Newton's Method is an iterative method to find a root of a function. For a function $f$, this would be to look for a value $x$ where $f(x) = 0$. It is a fixed point iteration that has a quadratic order of convergence. The fixed point iteration is on the form $x^{(k+1)}=x^{(k)} + \alpha f(x^{(k)})$ where for it to be Newton's Method $a = -f'(x^{(k)})^{-1}$. This would essentially mean that you are to divide $x^{(k)}$ by the derivative of itself to get the second term in the iteration.

# Newton's Method for Vector Nonlinear Equation
Newton's Method for Vectors is an iterative method to find solutions for $f(x) = 0$ in a nonsingular system of linear equations. There is no guarantee to find an answer as there is for linear systems using this method, but it can still be used in the cases that there exists one (or can be used to figure out IF there exists one. Similar to scalars, the fixed point iteration is on the form $x^{(k+1)}=x^{(k)} +Af(x^{(k)})$ where $A = -f'(x^{(k)})^{-1}$. $f'(x)$ for a system of equations is stated as a Jacobian, which is essentially all the different derivatives for the different variables in each separate equation in the vector function. The Jacobian matrix is on the form

$f'(x) = J = \begin{bmatrix}
\frac{df_1(x)}{dx_1(x)} & ... & \frac{df_1(x)}{dx_n(x)}\\
... & ... & ...\\
\frac{df_n(x)}{dx_1(x)} & ... & \frac{df_n(x)}{dx_n(x)}\\
\end{bmatrix}$



# **Method**

# Jacobi Iteration
To figure out an implementation of Jacobi Iteration we use both information about Richardson Iteration and Jacobi Iteration. The method that was decided to be used is loosely based upon the pseudocode from Algorithm 7.1 in the course book, but with the component form $x_i^{(k+1)} = a_{ii}^{-1} (b_i - \sum_{j\neq i} a_{ij} x_j^{(k)}) \forall i$ used in the main loop of the iteration taken from examle 7.8 in the course book.

In [50]:
def jacobi_iteration(A:np.ndarray, b:np.ndarray):
    n = b.size
    x = np.zeros(n)
    while np.linalg.norm(A @ x - b) > 10**-15:
        for i in range(n):
            total = 0
            for j in range(n):
                if j == i:
                    continue
                total += A[i, j] * x[j]
            x[i] = (1/A[i, i]) * (b[i] - total)
    return x


ji_A = np.array([[2, -1], [-1, 2]])
ji_b = np.array([1, 2])

print(jacobi_iteration(ji_A, ji_b))

[1.33333333 1.66666667]


# Gauss-Seidel Iteration

The Gauss-Seidel Iteration implementation is very similar to the Jacobi Iteration implementation, just with using a different splitting and a different component form. This time, $x_i^{(k+1)} = a_{ii}^{-1} (b_i - \sum_{j< i} a_{ij} x_j^{(k+1)} - \sum_{j> i} a_{ij} x_j^{(k+1)}) \forall i$, which is an equation taken from Example 7.9 in the course book. Another difference is the splitting being based upon a lower triangular matrix meaning we need a method to invert it. This is done using forward substitution trying to figure out the solution to $Lx_i = I_{:,i}$ for all $0 ... n$ where $x_i$ is the $i$:th column in the inverted matrix. The forward substitution is based upon pseudocode from Algorithm 5.1 in the course book.

In [51]:
def forward_substitution_inverter(L:np.ndarray):
    n = L.shape[0]
    identity = np.identity(n)
    x = np.zeros(L.shape)
    for c in range(n):
        b = identity[:, c]
        x[0, c] = b[0] / L[0, 0]
        for i in range(n):
            sum = 0
            for j in range(i):
                sum += L[i, j] * x[j, c]
            x[i, c] = (b[i] - sum) / L[i, i]
    return x


def gauss_seidel(A:np.ndarray, b:np.ndarray):
    n = b.size
    x = np.zeros(n)
    atri = np.tril(A)
    atriinv = forward_substitution_inverter(atri)
    while np.linalg.norm(A @ x - b) > 10**-15:
        for i in range(n):
            total = 0
            for j in range(n):
                if j == i:
                    continue
                total += A[i, j] * x[j]
            x[i] = (atriinv[i, i]) * (b[i] - total)
    return x


gs_A = np.array([[2, -1], [-1, 2]])
gs_b = np.array([1, 2])

print(gauss_seidel(gs_A, gs_b))

[1.33333333 1.66666667]


# Newton's Method for Scalar Nonlinear Equation

This implementation of Newton's method is based upon the information in chapter 8.3 in the course book, and specifically based on the pseudocode in Algorithm 8.2. It performs the equation defined in the introduction on an arbitrary function, in this example $3x^3 - 2x^2 - x$ which has roots at $-\frac{1}{3}, 0$ and $1$. To calculate the derivative I have implemented my own method based upon the central difference formula, which will try a bigger and bigger value of h until it finds a good derivative (the smaller the better but sometimes too small does not work).

In [52]:
def a_nonlinear_function(x):
    return 3*x**3 - 2*x**2 - x  # The Function 3x^3 - 2x^2 - x which has roots at 0, 1 and -1/3


def derivative(f, x):
    h = 10**-15
    for i in range(6):
        df = (f(x + h) - f(x - h)) / (2 * h)
        if df != 0:
            return df
        h *= 100
    raise ValueError("Derivative is zero, x already found.")


def newtons_method(f, x0):
    x = x0
    func_val = f(x)
    while abs(func_val) > 10**-15:
        x = x - func_val / derivative(f, x)
        func_val = f(x)
    return x


print(newtons_method(a_nonlinear_function, 20))

1.0


# Newton's Method for Vector Nonlinear Equation

This implementation of Newton's Method for vectors is based upon the information in chapter 8.4 in the course book. Most notably, the main function is based on algorithm 8.4 from the same chapter. To solve the problem, we also had to bring in some help from the previous assignment, this being the direct solver that used QR Factorization to solve a problem $Ax = b$. We also had to create a function that calculates the derivatives in the jacobian matrix, which is done by using the forward difference formula on the specific value we are interested in.

In [53]:
def a_nonlinear_system(x:np.ndarray):
    a = np.array([np.sin(x[1] + 2*x[0]), np.cos(x[0]) + 0.5])
    return a


def jacobian(f, x:np.ndarray):
    h = 10**-8
    n = x.size
    J = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            f_old_x = f(x)[i]
            x[j] += h
            df = f(x)[i] - f_old_x
            J[i, j] = df / h
    return J


def newtons_method_system(f, x0:np.ndarray):
    x = x0
    func_vector = f(x)
    while np.linalg.norm(func_vector) > 10**-6:
        Df = jacobian(f, x)
        dx = direct_solver(Df, -func_vector)
        x = x + dx
        func_vector = f(x)
    return x


print(newtons_method_system(a_nonlinear_system, np.array([1, 1.5])))

[ 2.09439512 -1.04719753]


# **Results**

In this section tests will be performed to verify that the solutions are correct

# Jacobi Iteration
To test the Jacobi Iteration we start by generating a random square matrix of size $n \times n$. Since all the values are between $[0, 1]$ we also add the identity matrix to it to make sure that it is diagonally dominant. We then generate a vector that is our $b$. We now check two residuals to see whether we achieve the correct answer. First, we test the residual $||Ax - b||$. We also manufacture our own solution $b = Ay$, which we test in the residual $||x_y - y||$ to see whether the real solution matches the generated one. These should both approach zero, and for this code they do.

In [54]:
def ji_test(iters):
    for test in range(iters):
        n = np.random.randint(2, 10)
        matrix = np.random.rand(n, n) + np.identity(n)
        vector = np.random.rand(n)

        x = jacobi_iteration(matrix, vector)
        res_axb = np.linalg.norm(matrix @ x - vector)

        y = np.random.rand(n)
        x_y = jacobi_iteration(matrix, matrix @ y)
        res_xy = np.linalg.norm(x_y - y)
        print(f"Test {test}: ||Ax-b|| = {res_axb}, ||x-y|| = {res_xy}")


ji_test(5)


Test 0: ||Ax-b|| = 1.1443916996305594e-16, ||x-y|| = 2.2887833992611187e-16
Test 1: ||Ax-b|| = 9.42898409468251e-16, ||x-y|| = 9.41467529262489e-16
Test 2: ||Ax-b|| = 5.551115123125783e-17, ||x-y|| = 3.1401849173675503e-16
Test 3: ||Ax-b|| = 7.043575467574348e-16, ||x-y|| = 3.79805414435608e-15
Test 4: ||Ax-b|| = 3.5652682413747997e-16, ||x-y|| = 5.4672143489065705e-16



# Gauss-Seidel Iteration

To test the implementation of Gauss-Seidel Iteration, we use the exact same test cases as for Jacobi Iteration. Info can be found in the section above.

In [55]:
def gs_test(iters):
    for test in range(iters):
        n = np.random.randint(2, 10)
        matrix = np.random.rand(n, n) + np.identity(n)
        vector = np.random.rand(n)

        x = gauss_seidel(matrix, vector)
        res_axb = np.linalg.norm(matrix @ x - vector)

        y = np.random.rand(n)
        x_y = gauss_seidel(matrix, matrix @ y)
        res_xy = np.linalg.norm(x_y - y)
        print(f"Test {test}: ||Ax-b|| = {res_axb}, ||x-y|| = {res_xy}")


gs_test(5)

Test 0: ||Ax-b|| = 5.721958498152797e-16, ||x-y|| = 7.301351988545593e-16
Test 1: ||Ax-b|| = 9.805224261780596e-16, ||x-y|| = 4.002966042486721e-16
Test 2: ||Ax-b|| = 5.117875266520903e-16, ||x-y|| = 6.391696875268275e-16
Test 3: ||Ax-b|| = 9.852251986074894e-16, ||x-y|| = 5.324442579404919e-16
Test 4: ||Ax-b|| = 6.933340566559918e-16, ||x-y|| = 4.150409932595612e-15



# Newton's Method for Scalar Nonlinear Equation

To test that the Newton's method works we first select a root and then create a non-linear function that will have a singular root which is at the point we selected. This is done by creating a parabola at the form $(x - root)^2$. We then generate a random first try for x0 and run the method. The output will then be put into the parabola and compared to the created root to see if they align.

In [56]:
def nm_test(iters):
    for test in range(iters):
        root = np.random.rand() * 200 - 100

        def f(x):
            return (x - root)**2

        x0 = np.random.rand() * 200 - 100
        xs = newtons_method(f, x0)
        res_f = abs(f(xs))
        res_xy = abs(xs - root)
        print(f"Test {test}: Root = {root}, |f(x)| = {res_f}, |x-y| = {res_xy}")


nm_test(5)

Test 0: Root = 2.566930617745385, |f(x)| = 7.0430861504069995e-16, |x-y| = 2.6538813369114678e-08
Test 1: Root = -71.24788019813548, |f(x)| = 3.4171874323933246e-16, |x-y| = 1.8485636132936634e-08
Test 2: Root = 92.77561525014494, |f(x)| = 3.482673247347898e-16, |x-y| = 1.8661921785678715e-08
Test 3: Root = -84.2099627684415, |f(x)| = 7.862475079682518e-16, |x-y| = 2.8040105348736688e-08
Test 4: Root = 95.30452190181441, |f(x)| = 7.096969104250101e-16, |x-y| = 2.6640137207323278e-08


# Newton's Method for Vector Nonlinear Equation

To test that the Newton's method works we create a nonlinear system of equations with only one solution. The selected test system is a system with one circle of radius 1 ($(x-offset)^2 + y^2 - 1 = 0$) and one $x=y$ line with a single tangent. This is offset in the $x$ axis by a random variable.

In [57]:
def nms_test(iters):
    for test in range(iters):
        offset = np.random.rand() * 2 - 1
        root = np.array([-math.cos(math.pi/4) + offset, math.cos(math.pi/4)])

        def f(x: np.ndarray):
            a = np.array([(x[0] - offset)**2 + (x[1])**2 - 1, x[0] - x[1] + math.sqrt(2) - offset])
            return a

        xs = newtons_method_system(f, np.array([np.random.rand(), np.random.rand()]))
        res_f = np.linalg.norm(f(xs))
        res_xy = np.linalg.norm(xs - root)
        print(xs)
        print(f"Test {test}: Offset = {offset} ||f(x)|| = {res_f}, ||x-y|| = {res_xy}")


nms_test(5)

[-0.72399225  0.70759804]
Test 0: Offset = -0.017376727842931095 ||f(x)|| = 4.826735251484138e-07, ||x-y|| = 0.0006947470943838688
[-0.84520346  0.70755102]
Test 1: Offset = -0.13854092484979041 ||f(x)|| = 3.9470268498043026e-07, ||x-y|| = 0.0006282536787710146
[-1.15480138  0.70774003]
Test 2: Offset = -0.4483278499703831 ||f(x)|| = 8.02010061251579e-07, ||x-y|| = 0.0008955501441386554
[-0.40546402  0.70758185]
Test 3: Offset = 0.3011676884449985 ||f(x)|| = 4.5138933790767055e-07, ||x-y|| = 0.0006718551463542368
[-0.13102981  0.70757875]
Test 4: Offset = 0.5756050046388395 ||f(x)|| = 4.4552079140087056e-07, ||x-y|| = 0.0006674678876142447


# **Discussion**

The methods this week required a bit more experimentation than the ones created last week, though it made them feel more like one's own work instead of just a rewriting of some pseudocode. The Newton's Method for Vector Nonlinear Equation took much more work to implement than I had first expected, but once a way was found to create it it was incredibly satisfying.

When it comes to the test, we can see that not everything is quite as good as it was last week. All tests are obviously showing something approaching zero but it is not as clear as it could have been. This is because of the processing limitations of Google Colab. Using much higher precision would take a lot more time, which Colab does not like. In any case, the results prove that the value would approach zero the more time and precision you throw at it.